In [1]:
from binance.um_futures import UMFutures
from binance.spot import Spot
import numpy as np
import pandas as pd
import matplotlib as plot
from datetime import datetime, timedelta
import math

In [22]:
# config_logging(logging, logging.DEBUG)
um_futures_client = UMFutures(timeout = 5)
spot_client = Spot(timeout = 5)
start_time_date = datetime(2020, 1, 1)
fr_raw = []
klines_raw = []
spot_raw = []
# a =  start_time_date+timedelta(hours=1000*8)
while start_time_date < datetime.now():
    start_time_ts = datetime.timestamp(start_time_date)
    fr_raw += um_futures_client.funding_rate(symbol = "ETHUSDT", startTime = int(start_time_ts)*1000, limit = 1000)
    klines_raw += um_futures_client.klines(symbol = "ETHUSDT", interval = "8h", startTime = int(start_time_ts)*1000, limit = 1000)
    spot_raw += spot_client.klines(symbol = "ETHUSDT", interval = "8h", startTime = int(start_time_ts)*1000, limit = 1000)
    start_time_date += timedelta(hours=1000*8)


In [23]:
fr_df = pd.DataFrame(fr_raw)
fr_df['fundingTime'] = list(map(lambda x: datetime.fromtimestamp(x/1000), fr_df['fundingTime']))

headers = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_vol', '#trade', 'taker_buy_vol', 'taker_buy_quote_vol', 'ignore']

spot_df = pd.DataFrame(spot_raw, columns = headers, dtype=float)

klines_df = pd.DataFrame(klines_raw, columns = headers, dtype=float)
# klines_df.drop(columns=['ignore', 'quote_vol', 'taker_buy_quote_vol'], inplace=True)
klines_df = klines_df.iloc[:, [0,1,2,4]]
klines_df['open_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000), klines_df['open_time']))
klines_df['s_open'] = spot_df['open']
klines_df['s_close'] = spot_df['close']
klines_df['funding_rate'] =  list(map(float, fr_df['fundingRate']))
klines_df['funding_rate'] =  list(map(lambda x: 1000*x, klines_df['funding_rate']))
klines_df['status'] = [False]*len(klines_df)
klines_df['hold'] = [0.0]*len(klines_df)
klines_df['short_position'] = [0.0]*len(klines_df)
klines_df['StopLoss'] = [0.0]*len(klines_df)
klines_df['s_pnl'] = [0.0]*(len(klines_df))
klines_df['p_pnl'] = [0.0]*(len(klines_df))
klines_df['fr_pnl'] = [0.0]*(len(klines_df))
klines_df['capital'] = [2000000.0]+[0.0]*(len(klines_df)-1)



In [27]:
capital = klines_df.loc[0,'capital']
spot_entry = 0.0
perp_entry = 0.0
sl = 0.0
fr_th = 0.2
flag = 0
for i in range(1, len(klines_df)):
    klines_df.loc[i,'status'] = klines_df.loc[i-1,'status']
    klines_df.loc[i,'hold'] = klines_df.loc[i-1,'hold']
    klines_df.loc[i,'short_position'] = klines_df.loc[i-1,'short_position']
    if klines_df.loc[i-1,'funding_rate'] >= fr_th and klines_df.loc[i-1,'hold'] == 0:
        # buy spot
        klines_df.loc[i,'hold'] = capital*0.95/2
        spot_entry = klines_df.loc[i,'s_open']
        # short perp
        klines_df.loc[i,'short_position'] = capital*0.95/2
        perp_entry = klines_df.loc[i,'open']
        capital -= (klines_df.loc[i,'hold']*0.01 + klines_df.loc[i,'short_position']*0.005)

        sl = perp_entry*1.9
        klines_df.loc[i,'status'] = True
    
    if klines_df.loc[i-1,'funding_rate'] < 0 and  klines_df.loc[i-1,'status']:
        klines_df.loc[i,'status'] = False
        klines_df.loc[(i),'s_pnl'] = (klines_df.loc[(i),'s_open']-spot_entry) / spot_entry*klines_df.loc[(i),'hold']
        klines_df.loc[(i),'p_pnl'] = -1*(klines_df.loc[(i),'open']-perp_entry) / perp_entry*klines_df.loc[(i),'short_position']
        klines_df.loc[i,'hold'] = 0
        klines_df.loc[i,'short_position'] = 0
        capital += klines_df.loc[(i),'s_pnl'] + klines_df.loc[(i),'p_pnl']
        flag = 1

    if klines_df.loc[i,'status']:
        # 空單爆倉
        if klines_df.loc[i,'high'] > sl:
            klines_df.loc[(i),'status'] = False
            klines_df.loc[(i),'s_pnl'] = (sl-spot_entry) / spot_entry*klines_df.loc[(i-1),'hold']
            klines_df.loc[(i),'p_pnl'] = -1*(sl-perp_entry) / perp_entry*klines_df.loc[(i-1),'short_position']
            capital -= (klines_df.loc[(i-1),'hold']*0.01 + klines_df.loc[(i-1),'short_position']*0.005)
            klines_df.loc[(i),'hold'] = 0
            klines_df.loc[(i),'short_position'] = 0
            klines_df.loc[(i),'StopLoss'] = 1
            flag = 1
        else:
            klines_df.loc[(i),'s_pnl'] = (klines_df.loc[(i),'s_close']-spot_entry) / spot_entry*klines_df.loc[(i),'hold']
            klines_df.loc[(i),'p_pnl'] = -1*(klines_df.loc[(i),'close']-perp_entry) / perp_entry*klines_df.loc[(i),'short_position']
            klines_df.loc[(i),'fr_pnl'] = klines_df.loc[(i),'short_position'] * klines_df.loc[(i),'funding_rate']/100
            capital += klines_df.loc[(i),'fr_pnl']

    if flag:
        klines_df.loc[(i),'capital'] =  capital
        flag = 0
    else:    
        klines_df.loc[(i),'capital'] =  capital + klines_df.loc[(i),'s_pnl'] + klines_df.loc[(i),'p_pnl'] 


# klines_df


現貨 0.1%
期貨 maker 0.02% taker 0.04%

每次開關單手續費成本 (0.1+0.04)*2 = 0.28%
一般正常資費0.01%

x*t > 0.28 
x 為資費門檻
t 為套利次數


In [28]:
# 計算每日報酬率
klines_df['策略每日報酬率'] = klines_df['capital'].pct_change()
klines_df.loc[0, '策略每日報酬率'] = 0  # 初始化第一天策略每日報酬率

# 計算累積報酬率
klines_df['策略累積報酬率'] = (1 + klines_df['策略每日報酬率']).cumprod()-1
klines_df.loc[0, '策略累積報酬率'] = 0  # 初始化第一天策略累積報酬率

# 計算Drawdown
# 計算每個時間點的最大價格
klines_df['每個時點最大策略累積資金(金額)'] = klines_df['capital'].cummax()
# 計算每個時間點的 Drawdown
klines_df['Drawdown'] = (klines_df['capital'] - klines_df['每個時點最大策略累積資金(金額)']) / capital

klines_df.head(5)

,open_time,open,high,close,s_open,s_close,funding_rate,status,hold,short_position,StopLoss,s_pnl,p_pnl,fr_pnl,capital,策略每日報酬率,策略累積報酬率,每個時點最大策略累積資金(金額),Drawdown
0,2020-01-01 00:00:00,129.61,130.51,129.12,129.71,129.16,0.10000,False,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,0.0,2000000.0,0.0
1,2020-01-01 08:00:00,129.12,130.97,130.17,129.16,130.24,0.10000,False,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,0.0,2000000.0,0.0
2,2020-01-01 16:00:00,130.15,132.49,131.97,130.24,132.08,0.07354,False,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,0.0,2000000.0,0.0
3,2020-01-02 00:00:00,132.00,132.96,130.62,132.08,130.77,-0.16125,False,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,0.0,2000000.0,0.0
4,2020-01-02 08:00:00,130.63,130.75,129.28,130.72,129.26,0.05047,False,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,0.0,2000000.0,0.0


In [29]:
from plotly.subplots import make_subplots  #載入plotly畫子圖的套件
import plotly.graph_objs as go  # 載入plotly套件
# 定義繪製策略損益圖方法
def draw_PnL(trace1, trace2, trace3,  title1, title2, title3):
    
    # 打開子畫布
    fig = make_subplots(rows=3, cols=1, subplot_titles=(title1, title2, title3), row_heights=[0.6, 0.25, 0.6], shared_xaxes=True, vertical_spacing=0.1)
    
    # 帶入資料到子畫布的特定位置
    fig.add_trace(trace1, row=1, col=1)
    fig.add_trace(trace2, row=2, col=1)
    fig.add_trace(trace3, row=3, col=1)
    # 設定子畫布共同屬性
    fig.update_layout(
        title='績效', # 設定圖表標題名稱
        plot_bgcolor='white', # 設定底色為白色
    )
    
    # 設定子畫布屬性
    fig.update_xaxes(row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="損益(%)", row=1, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')
    fig.update_xaxes(title_text="日期", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="MDD(%)", row=2, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')
    fig.update_xaxes(title_text="日期", row=3, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey')
    fig.update_yaxes(title_text="FR(%)", row=3, col=1, linecolor='black', linewidth=2, showgrid=True, gridcolor='lightgrey', zerolinecolor='darkgrey')

    #展示圖表
    fig.show()
    
    return
# 建立plotly損益曲線
trace1 = go.Scatter(x=klines_df['open_time'], 
                    y=klines_df['策略累積報酬率']*100, 
                    mode='lines', 
                    name='損益', 
                    line=dict(color='blue')
                    )
trace2 = go.Scatter(x=klines_df['open_time'], 
                    y=klines_df['Drawdown']*100, 
                    mode='lines', 
                    name='Drawdown', 
                    line=dict(color='red')
                    )

trace3 = go.Scatter(x=klines_df['open_time'], 
                    y=klines_df['funding_rate'], 
                    mode='lines', 
                    name='funding_rate', 
                    line=dict(color='black')
                    )

# 繪製策略損益圖
# 計算績效指標
total_return = klines_df.loc[len(klines_df)-1, '策略累積報酬率']
annualized_return = (1+klines_df.loc[len(klines_df)-1, '策略累積報酬率'])**(365/3/len(klines_df))-1  # 年化報酬率
annualized_volatility = np.std(klines_df['策略每日報酬率'])*np.sqrt(365)  # 年化波動率
sharpe_ratio = annualized_return/annualized_volatility  # 年化夏普值
max_drawdown = klines_df['Drawdown'].min()  # 最大回撤(MDD)

print("總報酬率: {}%".format(round(total_return,2)))
print("年化報酬率: {}%".format(round(annualized_return,2)))
print("年化波動率: {}%".format(round(annualized_volatility*100,2)))
print("年化夏普值: {}".format(round(sharpe_ratio,2))) 
print("最大回撤(MDD): {}%".format(round(max_drawdown*100,2)))
print(f"爆倉次數: {sum(klines_df['StopLoss'])}")
draw_PnL(trace1, trace2, trace3, '損益曲線', 'Drawdown', 'FR')

總報酬率: 920.47%
年化報酬率: 14.64%
年化波動率: 2.18%
年化夏普值: 6.72
最大回撤(MDD): -1.46%
爆倉次數: 8.0
